Miglioriamo l'analisi
====

Miglioriamo l'analisi andando ad aggiungere 

* packed padded sequences
* pre-trained word embeddings
* una architettura RNN differente
* bidirectional RNN
* multi-layer RNN
* regolarizzazione
* un ottimizzatore differente


Prima di eseguire qualsiasi altro passo andiamo ad impostare il seed per ottenere sempre gli stessi risultati

In [1]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Preparare i dati
----

Andiamo a gestire le packed padded sequences, questo permette alla RNN di processare solo gli elementi non ```<pad>``` e per gli elementi ```<pad>``` andremo a caricare un tensore a 0. Per far questo dobbiamo dire alla RNN quanto la nostra sequenza è lunga. Per farlo basta impostare  ```include_lengths = True``` per il nostro campo TEXT. ora ```batch.text``` diventerà una tupla con il primo elemento la nostra sequenza e come secondo elemento la lunghezza della stessa.

In [2]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

Carichiamo il dataset

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

andiamo a creare il dataset di validation

In [4]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Ora andiamo a caricare il word embeddings, pre-trained. Invece che avere il nostro strato di embeddings andiamo ad utilizzare una rappresentazione della parola pre-trained. Per farlo basta semplicemente dire che vettore vogliamo caricare e passarlo come argomento a build_vocab. TorchText scaricherà per noi i vettori e associerà ad ogni parola del dizionario il vettore corretto del nostro dizionario.

Useremo il "glove.6B.100d". Glove è un algoritmo di vettorializzazione 6B sta ad indicare che i vettori sono stati addestrati su 6 Miliardi di token e hanno una dimensionalità di 100

Per vedere gli altri vettori disponibili basta andare [qui](https://github.com/pytorch/text/blob/master/torchtext/vocab.py#L146).

In questo spazio vettoriale le parole con un significato simile sono adiacenti, ad esempio "terrible", "awful", "dreadful" sono posizionate vicine. Questo è una buona cosa in quanto il modello non deve imparare queste relazioni da zero.

**Attenzione** questi vettori hanno una dimensione di circa 862MB.

Di default, TorchText inizializzerà le parole nel nostro vocabolario con un vettore a 0, noi non vogliamo questo andremo ad impostare il token UNK randomicamente impostando ```unk_init``` con ```torch.Tensor.normal_```.

In [5]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

Come prima, creiamo gli iteratori e piazziamo i tensori nella GPU se questa è disponibile. 
Unl'altra cosa per gestire le frasi con packed padded, tutti i tensori dentro al batch devono essere ordinati per la loro lunghezza. Questo viene gestito con il parametro ```sort_within_batch = True``` nell'iteratore. 




In [6]:

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#per getire il problema con win10 e i driver cuda commentare prima di fare la push
#device = 'cpu'

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

Costruzione del modello
----

Il modello che andremo a costruire è molto differente dal precedente

Architettura LSTM
----

Perchè la rete LSTM è migliore di una RNN ? Le reti RNN soffrono del problema del vanishing gradient.
Le reti LSTM superano questo problema con uno strato di memoria extra chiamato cell. **TODO: spiegare meglio**

<img src="./images/lstm.png" />

Bidirectional RNN
----

Il concetto dietro una RNN Bidirezionale è semplice. Abbiamo una RNN che processa i tokens in un verso (forward) e una RNN che processa gli stessi nel verso opposto (backward).
In pyhtorch i tensori hidden state e backward sono impilati uno sopra l'altro in un singolo tensore.

<img src="./images/bidirectional.png" />

Multi Layer RNN
----

Le reti multilayer (dette anche deep RNN) sono un concetto abbastanza semplice.
L'idea è che possiamo aggiungere una RNN addizionale sopra un'altra RNN. L'idea è che l'output (hidden state) della rete più in basso diventa l'input della rete più in alto. 

<img src="./images/multilayer.png" />

Regolarizzazione
----

Sebbene abbiamo aggiunto molti miglioramenti al modello, abbiamo anche aggiunto molti parametri addizionali.Esiste il rischio concreto di overfitting. Per combattere questo introduciamo la regolarizzazione nel nostro caso il dropout.

Implementazione
----

Un'altro miglioramento al modello è che non andremo ad imparare dal token pad, andando esplicitamente a dire al modello che tali token sono irrilevanti per determinare il sentimento di una sequenza.

La rete LSTM non ritorna semplicemente uno stato hidden ma bensi uno stato di output, e una tupla di stati hidden e cell.
Lo stato finale della nostra LSTM, sia la componente forward che la componente backward vengono concatenate assieme (Sicchè l'ingresso della f.c. finale sarà il doppio dello hidden state).

Per implementare la bidirezionalità e aggiungere layer addizionali vanno usati i parametri ```num_layers``` e ```bidirectional```.

Per poter utilizzare il packed padded sequences, dobbiamo aggiungere il parametro ```text_lenght``` a forward. Prima di passare il tensore di embeddings alla RNN dobbiamo impacchettarlo (Pack), questo farà si che la nostra rete analizzerà solo i token non pad.

La RNN restituisce packed_output e la tupla hidden e cell.

Senza la gestione del pad l'ultimo stato si sarebbe riferito quasi sicurtamente al pad. Usiamo la funzione ```unpack``` per andare ad ottenere di nuovo il tensore alla dimensione originale. Gli elementi padding di output vengono riportati a 0, viene fatto se si utilizzare di nuovo la sequenza più avanti nel modello.

Lo stato finale ha una dimensionalità **[num layers * num directions, batch size, hid dim]** e i layer sono ordinati in questo modo **[forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]**.

Se vogliamo il layer finale forward e backward dobbiamo prendere il penultimo e ultimo dalla prima dimensione e concatenarlo.


In [7]:

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #dimensione del batch text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #ora output ha la dimensione originale
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concateno il layer  forward finale (hidden[-2,:,:]) e il backward (hidden[-1,:,:]) 
        #e gli applico il droput
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

Come prima ci andiamo a instanziare il modello impostando i nuovi parametri
Dobbiamo impostare la dimensionalità di embedding uguale a quella dei vettori GloVe caricati prima.
Andiamo anche ad estrarre l'indice del token  ```<pad> ```.

In [8]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

Andiamo a vedere il numero dei parametri del modello, notiamo che ora sono più del doppio.

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


Ora dobbiamo copiare il pre-trained word embeddings dentro lo strato di embedding del modello.
Controlliamo anche la dimensionalità di embedding che deve essere **[vocab size, embedding dim]**.

In [10]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


Andiamo a copiare il tensore all'interno di embeddings.

**NOTA** Questo deve essere fatto usando weight.data e non weight

In [11]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1386,  0.1180,  0.3534,  ...,  0.1226,  0.5973, -0.1702],
        [-0.0786,  0.0541, -0.0993,  ...,  0.2565, -0.1874, -0.4428],
        [-0.3617,  0.6201,  0.1105,  ...,  0.2994, -0.5920,  1.0949]])

Andiamo ad azzerare le posizioni di UNK e PAD

In [12]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1386,  0.1180,  0.3534,  ...,  0.1226,  0.5973, -0.1702],
        [-0.0786,  0.0541, -0.0993,  ...,  0.2565, -0.1874, -0.4428],
        [-0.3617,  0.6201,  0.1105,  ...,  0.2994, -0.5920,  1.0949]])


Andiamo ad impostare l'ottimizzatore e il criterio

Train del modello
----

L'unico cambiamento che andiamo a fare è relativo all'ottimizzatore invece che **SDG** andiamo ad usare **ADAM** [qui](https://ruder.io/optimizing-gradient-descent/index.html) una guidina sui vari otimizzatori

In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

Recuperiamo anche la funzione per calcolare l'accuratezza

In [14]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

Definiamo la funzione di trainig per il nostro modello.

Siccome abbiamo impostato include_lengths = True il nostro batch.text ora è una tupla con il primo elemento la lista dei vari token tradotti in numeri e il secondo elemento il numero di token nella prima lista. Separiamo il risultato in due variabili che passeremo al modello 

**Nota:** Ora stiamo usando il droput, dobbiamo assicurarci che sia abilitato quando andiamo a fare il train usando il metodo model.train()

In [15]:

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

ora andiamo a fare la valutazione del modello model.eval() spegne il dropout 

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

**Nota:** con windows 10 ci sono problemi con le reti lstm ho disabilitato il train, per riabilitarlo commentare la prima riga

In [18]:
#%%script false 
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Couldn't find program: 'false'


ora andiamo a caricare il modello e valutiamo il risultato

In [20]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.973 | Test Acc: 57.88%


Input personalizzato
----

Possiamo utilizzare il modello per predire il sentimento su una nostra nuova frase.
Usiamo il modello in evaluation mode. La nostra funzione predict_sentiment fa le seguenti attività:

* imposta il modello in evaluation mode
* tokenizza la frase in ingresso andando a spezzare la stringa in una lista di token
* prende i vari token e li trasforma nella rappresentazione numerica andando a utilizzare il nostro vocabolario
* legge la lunghezza della frase
* converte la rappresentazione numerica in una lista di tensori
* aggiungo una dimensione unsqueezeing per poter farla gestire a pytorch
* converte la lunghezza in un tensore
* spalmo il risultato del risultato del modello tra 0 e 1 tramite la funzione di sigmoid
* converto il tensore in un intero tramire la funzione item()
    

Ora per le recensioni negative ci aspettiamo che il modello ritorni un valore vicino allo 0 e per le recensioni positive ci venga restituito un valore vicino a 1.

In [21]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

andiamo ad analizzare un esempio di recensione negativa

In [22]:
predict_sentiment(model, "This film is terrible")

0.7027969360351562

un esempio di recensione positiva

In [23]:
predict_sentiment(model, "This film is great")

0.8941178917884827